# Statistical Analysis of the Fraud Detection Project

In this notebook, we perform descriptive analytics and feature exploration on transactional data stored in the Silver layer of our Fabric lakehouse. The goal is to understand customer behavior, identify patterns and engineer features that could be used to flag anomalous or potentially fraudulent transactions.

In [23]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/Tables/Silver/bank_transactions_data/part-00000-5ee6fa47-1fec-453c-af80-50954400a764-c000.snappy.parquet"
df = pd.read_parquet("/lakehouse/default/Tables/Silver/bank_transactions_data/part-00000-5ee6fa47-1fec-453c-af80-50954400a764-c000.snappy.parquet")
display(df)


## Data Schema Overview

Below are the columns present in the dataset. These will be used to derive customer behavior metrics and potential anomaly indicators.


In [24]:
df.columns

Index(['TransactionID', 'AccountID', 'TransactionAmount', 'TransactionDate',
       'TransactionType', 'Location', 'DeviceID', 'IP_Address', 'MerchantID',
       'Channel', 'CustomerAge', 'CustomerOccupation', 'TransactionDuration',
       'LoginAttempts', 'AccountBalance', 'PreviousTransactionDate'],
      dtype='object')

## Behavioral Metrics

We'll now explore transaction patterns per customer to identify behavioral trends and outliers.

### 1. Transaction Frequency per Account

We begin by analyzing how many transactions each account has made. Accounts with unusually high activity could indicate high-risk behavior or bot-driven transactions.


In [25]:
transactions_per_customer = df.groupby('AccountID')['TransactionID'].count()
print(transactions_per_customer)

AccountID
AC00001    2
AC00002    7
AC00003    5
AC00004    9
AC00005    9
          ..
AC00496    3
AC00497    6
AC00498    8
AC00499    7
AC00500    4
Name: TransactionID, Length: 495, dtype: int64


### 2. Spending Profile per Account

Next, we calculate the average transaction amount and the standard deviation per account. This helps identify customers with erratic spending behavior or unusually high transaction variability.

In [26]:
transaction_stats = df.groupby('AccountID')['TransactionAmount'].agg(['mean', 'std']).reset_index()
transaction_stats.columns = ['AccountID', 'AverageAmount', 'AmountStdDev']
print(transaction_stats)

    AccountID  AverageAmount  AmountStdDev
0     AC00001     130.380000    116.799898
1     AC00002     293.744286    195.695091
2     AC00003     253.268000    158.484753
3     AC00004     242.231111    231.081586
4     AC00005     347.974444    189.487268
..        ...            ...           ...
490   AC00496     198.626667    141.862904
491   AC00497     197.868333    107.795494
492   AC00498     381.377500    423.171694
493   AC00499     331.612857    273.085811
494   AC00500     161.740000     44.404058

[495 rows x 3 columns]


Detects accounts with significant fluctuations in spending patterns.

### 3. Geographic Variability

High location volatility—frequent changes in the city or region of transactions—may suggest fraudulent activity such as account takeover or remote access.

We compute a `LocationChange` flag to capture whether a customer's transaction location differs from their previous transaction.


In [27]:
df['LocationChange'] = df.groupby('AccountID')['Location'].shift() != df['Location']
df['LocationChange'] = df['LocationChange'].astype(int)

print(df[['AccountID', 'Location', 'LocationChange']])

     AccountID          Location  LocationChange
0      AC00128         San Diego               1
1      AC00455           Houston               1
2      AC00019              Mesa               1
3      AC00070           Raleigh               1
4      AC00411           Atlanta               1
...        ...               ...             ...
2507   AC00297  Colorado Springs               1
2508   AC00322            Tucson               1
2509   AC00095         San Diego               1
2510   AC00118            Denver               1
2511   AC00009      Jacksonville               1

[2512 rows x 3 columns]


### 4. Temporal Anomalies – Rapid Transactions

Fraudulent activity often manifests as multiple transactions occurring in rapid succession, especially in compromised accounts or scripted attacks.

We'll calculate the time difference between consecutive transactions for each account and flag those that occur within a 10-second window.


In [28]:
df['TimeDifference'] = df.groupby('AccountID')['TransactionDate'].diff().dt.total_seconds()
df['FlaggedRapidTxns'] = df['TimeDifference'] < 10  # Less than 10 seconds
flagged_rapid_txns = df[df['FlaggedRapidTxns']]
print(flagged_rapid_txns[['TransactionID', 'AccountID', 'TransactionDate', 'TimeDifference']])

     TransactionID AccountID           TransactionDate  TimeDifference
33        TX000034   AC00359 2023-03-13 17:23:24+00:00     -23850530.0
72        TX000073   AC00282 2023-07-04 18:20:33+00:00     -13988465.0
79        TX000080   AC00014 2023-07-17 18:54:31+00:00      -2158620.0
88        TX000089   AC00438 2023-06-22 18:37:29+00:00      -9158828.0
94        TX000095   AC00425 2023-03-02 16:05:09+00:00     -21602594.0
...            ...       ...                       ...             ...
2502      TX002503   AC00069 2023-04-21 16:22:38+00:00     -19698116.0
2503      TX002504   AC00427 2023-11-06 16:56:41+00:00      -1556863.0
2505      TX002506   AC00377 2023-01-27 17:38:07+00:00      -2415326.0
2510      TX002511   AC00118 2023-02-24 16:24:46+00:00      -3889685.0
2511      TX002512   AC00009 2023-02-14 16:21:23+00:00       -518116.0

[975 rows x 4 columns]


### 5. Summary of Rapid Transactions

We now calculate the number and total value of all transactions flagged for happening in rapid succession. These metrics help quantify potential fraud exposure from this behavioral dimension.


In [29]:
# Count the total number of flagged fraudulent transactions
total_rapid_fraud_count = flagged_rapid_txns['TransactionID'].count()

# Sum of transaction amounts for flagged rapid transactions
total_rapid_fraud_amount = flagged_rapid_txns['TransactionAmount'].sum()

print(f"Total Flagged Rapid Transactions: {total_rapid_fraud_count}")
print(f"Total Amount of Flagged Rapid Transactions: ${total_rapid_fraud_amount:.2f}")

Total Flagged Rapid Transactions: 975
Total Amount of Flagged Rapid Transactions: $280539.04


### 6. Behavioral Anomaly Detection

This section focuses on identifying suspicious customer behaviors based on frequency, value and variability of transactions. These features serve as early indicators of potential fraud and provide key inputs for future anomaly detection models.


In [30]:
spending_frequency = df.groupby('AccountID')['TransactionID'].count()
print(spending_frequency)

AccountID
AC00001    2
AC00002    7
AC00003    5
AC00004    9
AC00005    9
          ..
AC00496    3
AC00497    6
AC00498    8
AC00499    7
AC00500    4
Name: TransactionID, Length: 495, dtype: int64


### 7. Accounts with Unusually High Transaction Volume

We calculate the mean and standard deviation of transaction counts per account. Any account exceeding 3 standard deviations above the mean is flagged as high-frequency and potentially suspicious.


In [31]:
avg_transactions = spending_frequency.mean()
std_transactions = spending_frequency.std()

# Define a threshold to flag accounts with unusually high transaction counts
threshold = avg_transactions + 3 * std_transactions  # 3 standard deviations above the mean

# Flag accounts with transaction count above threshold
high_spending_accounts = spending_frequency[spending_frequency > threshold]

print(f"Accounts with unusually high transaction counts: {high_spending_accounts}")

Accounts with unusually high transaction counts: AccountID
AC00202    12
AC00362    12
AC00363    12
AC00460    12
Name: TransactionID, dtype: int64


### 8. Total Spending from High-Activity Accounts

Now we compute the total transaction value for these flagged accounts. High value combined with high frequency may indicate fraudulent behavior.


In [32]:
# Get the total spending for high spending accounts
high_spending_transactions = df[df['AccountID'].isin(high_spending_accounts.index)]

# Total amount spent in flagged high spending accounts
total_high_spending = high_spending_transactions['TransactionAmount'].sum()

print(f"Total spending for accounts with unusually high transaction counts: ${total_high_spending:.2f}")

Total spending for accounts with unusually high transaction counts: $16986.69


### 9. Transaction Amount Z-Score Analysis

We apply a Z-score transformation to identify transactions that deviate significantly from the average spending pattern. Transactions beyond 3 standard deviations (Z-score > 3 or < -3) are flagged.


In [33]:
# Calculate Z-score for transaction amounts
df['TransactionZScore'] = (df['TransactionAmount'] - df['TransactionAmount'].mean()) / df['TransactionAmount'].std()

# Flag transactions with Z-score > 3 as potential fraud
df['PotentialFraud'] = df['TransactionZScore'].abs() > 3

print(df[['TransactionID', 'TransactionAmount', 'TransactionZScore', 'PotentialFraud']])

     TransactionID  TransactionAmount  TransactionZScore  PotentialFraud
0         TX000001              14.09          -0.971082           False
1         TX000002             376.24           0.269386           False
2         TX000003             126.29          -0.586765           False
3         TX000004             184.50          -0.387379           False
4         TX000005              13.45          -0.973274           False
...            ...                ...                ...             ...
2507      TX002508             856.21           1.913422           False
2508      TX002509             251.54          -0.157747           False
2509      TX002510              28.63          -0.921278           False
2510      TX002511             185.97          -0.382344           False
2511      TX002512             243.08          -0.186725           False

[2512 rows x 4 columns]


In [34]:
# Filter transactions flagged as potential fraud
fraudulent_transactions = df[df['PotentialFraud'] == True]

# Display the fraudulent transactions
print(fraudulent_transactions[['TransactionID', 'AccountID', 'TransactionAmount', 'TransactionZScore', 'PotentialFraud']])

     TransactionID AccountID  TransactionAmount  TransactionZScore  \
74        TX000075   AC00265            1212.51           3.133852   
85        TX000086   AC00098            1340.19           3.571192   
176       TX000177   AC00363            1362.55           3.647782   
190       TX000191   AC00396            1422.55           3.853299   
274       TX000275   AC00454            1176.28           3.009753   
311       TX000312   AC00285            1221.65           3.165159   
340       TX000341   AC00107            1830.00           5.248933   
344       TX000345   AC00156            1271.90           3.337280   
375       TX000376   AC00316            1392.54           3.750506   
440       TX000441   AC00040            1237.56           3.219655   
475       TX000476   AC00464            1431.30           3.883270   
486       TX000487   AC00148            1416.69           3.833227   
535       TX000536   AC00161            1182.86           3.032292   
555       TX000556  

### 10. Fraudulent Transaction Summary

A quick overview of how many transactions were flagged using Z-score and the total dollar value involved.


In [35]:
# Count the total number of fraudulent transactions
total_fraud_count = df['PotentialFraud'].sum()

# Sum of transaction amounts for fraudulent transactions
total_fraud_amount = df.loc[df['PotentialFraud'] == True, 'TransactionAmount'].sum()

print(f"Total Fraudulent Transactions: {total_fraud_count}")
print(f"Total Amount of Fraudulent Transactions: ${total_fraud_amount:.2f}")

Total Fraudulent Transactions: 48
Total Amount of Fraudulent Transactions: $67404.68


### 11. IP Address Analysis

Shared or frequently changing IP addresses can indicate account sharing, bot activity, or compromised accounts.


In [36]:
ip_counts = df.groupby('IP_Address')['AccountID'].nunique()
suspicious_ips = ip_counts[ip_counts > 3].index  # More than 3 accounts using the same IP

df['Flag_SharedIP'] = df['IP_Address'].isin(suspicious_ips)
df[df['Flag_SharedIP']]

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP_Address,MerchantID,Channel,...,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,LocationChange,TimeDifference,FlaggedRapidTxns,TransactionZScore,PotentialFraud,Flag_SharedIP
1,TX000002,AC00455,376.24,2023-06-27 16:44:19+00:00,Debit,Houston,D000051,13.149.61.4,M052,ATM,...,141,1,13758.91,2024-11-04 08:09:35+00:00,1,NaN,False,0.269386,False,True
2,TX000003,AC00019,126.29,2023-07-10 18:16:08+00:00,Debit,Mesa,D000235,215.97.143.157,M009,Online,...,56,1,1122.35,2024-11-04 08:07:04+00:00,1,NaN,False,-0.586765,False,True
3,TX000004,AC00070,184.50,2023-05-05 16:32:11+00:00,Debit,Raleigh,D000187,200.13.225.150,M002,Online,...,25,1,8569.06,2024-11-04 08:09:06+00:00,1,NaN,False,-0.387379,False,True
4,TX000005,AC00411,13.45,2023-10-16 17:51:24+00:00,Credit,Atlanta,D000308,65.164.3.100,M091,Online,...,198,1,7429.40,2024-11-04 08:06:39+00:00,1,NaN,False,-0.973274,False,True
6,TX000007,AC00199,7.08,2023-02-15 16:36:48+00:00,Credit,Seattle,D000241,140.212.253.222,M019,ATM,...,139,1,13316.71,2024-11-04 08:10:09+00:00,1,NaN,False,-0.995093,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2504,TX002505,AC00047,123.38,2024-01-01 17:21:56+00:00,Debit,Sacramento,D000219,163.129.70.210,M049,Online,...,40,1,13542.41,2024-11-04 08:10:53+00:00,1,9932725.0,False,-0.596732,False,True
2508,TX002509,AC00322,251.54,2023-03-22 17:36:48+00:00,Debit,Tucson,D000410,49.174.157.140,M029,Branch,...,177,1,254.75,2024-11-04 08:11:42+00:00,1,1036670.0,False,-0.157747,False,True
2509,TX002510,AC00095,28.63,2023-08-21 17:08:50+00:00,Debit,San Diego,D000095,58.1.27.124,M087,Branch,...,146,1,3382.91,2024-11-04 08:08:39+00:00,1,17197370.0,False,-0.921278,False,True
2510,TX002511,AC00118,185.97,2023-02-24 16:24:46+00:00,Debit,Denver,D000634,21.190.11.223,M041,Online,...,19,1,1776.91,2024-11-04 08:12:22+00:00,1,-3889685.0,True,-0.382344,False,True


### 12. Long Duration Transactions

Transactions taking longer than usual (top 5% by duration) may represent failed attempts, bots retrying, or suspicious behavior.


In [37]:
# Identify transactions with unusually long durations
long_duration_threshold = df['TransactionDuration'].quantile(0.95)  # Top 5% as threshold
long_duration_transactions = df[df['TransactionDuration'] > long_duration_threshold]
print(f"Number of transactions with long duration (top 5%): {len(long_duration_transactions)}")
display(long_duration_transactions.head())

Number of transactions with long duration (top 5%): 123


### 13. High IP Variability per Account

Accounts logging in from many different IPs may be compromised or shared among multiple users.


In [38]:
# Frequent IP Changes for a Single Account
# Group by AccountID to count unique IP Addresses
ip_variability = df.groupby('AccountID')['IP_Address'].nunique().reset_index().rename(columns={'IP_Address': 'UniqueIPAddresses'})
# Identify accounts with a high number of unique IP Addresses
high_ip_variability_accounts = ip_variability[ip_variability['UniqueIPAddresses'] > 3]  # Example threshold
print(f"Number of accounts with high IP variability: {len(high_ip_variability_accounts)}")
display(high_ip_variability_accounts)

Number of accounts with high IP variability: 366


### 14. Shared IPs Across Multiple Accounts

IP addresses used by multiple distinct accounts may indicate coordinated or malicious behavior.


In [39]:
# Multiple Accounts Using the Same IP Address
ip_counts = df.groupby('IP_Address')['AccountID'].nunique().reset_index()
ip_counts.columns = ['IP_Address', 'UniqueAccounts']
shared_ips = ip_counts[ip_counts['UniqueAccounts'] > 1]
print(f"Number of IP addresses used by multiple accounts: {len(shared_ips)}")
display(shared_ips.head())

Number of IP addresses used by multiple accounts: 552


### 15. Merchant Risk Profiling

We analyze merchant-level transaction patterns to identify those with:

- High transaction variability
- Extremely large single transactions
- High transaction volumes

These can help us flag merchants potentially involved in fraudulent networks.


In [40]:
# Calculate merchant statistics
merchant_stats = df.groupby('MerchantID')['TransactionAmount'].agg(
    TotalTransactions='count',
    AverageTransactionAmount='mean',
    MaxTransactionAmount='max',
    StdTransactionAmount='std'
).reset_index()

# Define criteria for outliers
overall_std = df['TransactionAmount'].std()
overall_mean = df['TransactionAmount'].mean()

high_std_merchants = merchant_stats[merchant_stats['StdTransactionAmount'] > overall_std]
high_value_merchants = merchant_stats[merchant_stats['MaxTransactionAmount'] > (overall_mean * 3)]
high_transaction_count_merchants = merchant_stats[merchant_stats['TotalTransactions'] > merchant_stats['TotalTransactions'].quantile(0.95)]

# Combine criteria
outlier_merchants = pd.concat([high_std_merchants, high_value_merchants, high_transaction_count_merchants]).drop_duplicates()
# Visualize and analyze
print(f"Number of identified outlier merchants: {len(outlier_merchants)}")
display(outlier_merchants)

Number of identified outlier merchants: 66


### 16. Channel-Based Transaction Patterns

We review the distribution of transactions across different channels and transaction types to uncover platform-specific anomalies or misuse.


In [41]:
# Analyze fraud likelihood by channel and transaction type
channel_analysis = df.groupby(['Channel', 'TransactionType'])['TransactionID'].count().reset_index()
channel_analysis.columns = ['Channel', 'TransactionType', 'Count']
print(channel_analysis)

  Channel TransactionType  Count
0     ATM          Credit     73
1     ATM           Debit    760
2  Branch          Credit    251
3  Branch           Debit    617
4  Online          Credit    244
5  Online           Debit    567


### Conclusion: Key Insights from Statistical Analysis

This statistical analysis provided several meaningful insights into behavioral patterns, transactional anomalies and potential fraud signals within the dataset:

---

#### 1. Behavioral Red Flags
- **High-Frequency Accounts**: A small number of accounts exhibited significantly more transactions than the average user, flagging them as potential sources of automated or scripted behavior.
- **Rapid Transaction Bursts**: Some accounts made multiple transactions within seconds — a strong indicator of either bot activity or testing by malicious actors.
- **Shared Devices and IPs**: Multiple accounts were traced to the same IP addresses and devices, hinting at either shared access environments (e.g., fraud rings or testing setups) or compromised credentials.
- **Frequent IP Switching**: Certain users showed high IP variability, often linked to VPN usage or distributed attacks.

---

#### 2. Transaction-Based Anomalies
- **Outlier Transactions via Z-Score**: Several transactions had values far beyond the statistical norm, both on the low and high end, which may bypass rules-based systems without this statistical view.
- **Long Duration Transactions**: The longest transactions (top 5%) may suggest technical issues or deliberate manipulation, particularly when paired with high value or failed login attempts.
- **High-Value Accounts**: A subset of accounts exhibited both frequent and high-value spending — especially relevant for financial institutions focused on transaction risk scoring.

---

#### 3. Merchant Risk Signals
- **Outlier Merchant Behavior**: A few merchants stood out due to unusually high transaction values, high variability, or transaction volumes, making them worth further investigation for collusion or abuse.

---

#### 4. Channel and Transaction Type Trends
- The distribution of transaction types across different channels reveals how customers interact with the system. Any significant deviation from the norm can guide future channel-specific security protocols.

---
